In [ ]:


def get_wav_files_with_details(base_directory):
    data = []

    # Loop through all folders in the base directory
    for root, dirs, files in os.walk(base_directory):
        folder_name = os.path.basename(root)

        # Check if the folder name contains "cafe" or "white"
        if "cafe" in folder_name.lower() or "white" in folder_name.lower():
            print(folder_name)
            for file in files:
                # Check if the file is a .wav file
                if file.endswith(".wav"):
                    # Extract parts of the file name
                    parts = file.split('_')
                    audioname = '_'.join(parts[:3])  # Join the first three parts for audioname
                    attack_type = '_'.join(parts[3:]).replace('.wav', '')  # Join the remaining parts for attack_type
                    
                    # Include only files where attack_type contains "white" or "cafe"
                    if "white" in attack_type.lower() or "cafe" in attack_type.lower():
                        # Add to data
                        data.append({'audioname': audioname, 'attack_type': attack_type})
    
    # Create a DataFrame
    df = pd.DataFrame(data)
    return df

# Example usage
base_directory = "/data/ASV19/flac"  # Change this to your base directory path
wav_files_df = get_wav_files_with_details(base_directory)

# Print the DataFrame
print("Filtered DataFrame:")
print(wav_files_df)

# Optionally save to a CSV file
wav_files_df.to_csv("/data/ASV19/filtered_wav_files.csv", index=False)


Filtered DataFrame:
Empty DataFrame
Columns: []
Index: []


In [3]:
import os
import pandas as pd

In [4]:
def get_files(directory):
    flac_files = []
    
    # Walk through the directory
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.flac') or file.endswith('.wav') or file.endswith('.mp3'):
                flac_files.append( file)
    
    return flac_files

In [5]:
get_files("/data/ASV19/wav_paired_2")

['LA_T_1030482_RT09_recompression256k.wav',
 'LA_T_1021310_RT06_recompression64k.wav',
 'LA_T_1029007_RT09_recompression320k.wav',
 'LA_T_1000137_RT06_recompression64k.wav',
 'LA_T_1004644_RT03_recompression320k.wav',
 'LA_T_1021789_RT09_recompression16k.wav',
 'LA_T_1005349_RT09_recompression320k.wav',
 'LA_T_1000406_RT09_recompression320k.wav',
 'LA_T_1029184_RT06_recompression256k.wav',
 'LA_T_1021749_RT09_recompression256k.wav',
 'LA_T_1006715_RT06_recompression64k.wav',
 'LA_T_1000648_RT03_recompression196k.wav',
 'LA_T_1029621_RT06_recompression320k.wav',
 'LA_T_1021790_RT06_recompression320k.wav',
 'LA_T_1006969_RT09_recompression64k.wav',
 'LA_T_1000824_RT03_recompression256k.wav',
 'LA_T_1029929_RT09_recompression256k.wav',
 'LA_T_1022109_RT03_recompression196k.wav',
 'LA_T_1007217_RT09_recompression256k.wav',
 'LA_T_1001074_RT03_recompression320k.wav',
 'LA_T_1007408_RT09_recompression320k.wav',
 'LA_T_1022551_RT03_recompression256k.wav',
 'LA_T_1030559_RT09_recompression256k

In [7]:
import pandas as pd

def process_file_names(file_names):
    data = []

    # Loop through each file name
    for file in file_names:
        # Remove the file extension (.wav)
        file = file.replace(".wav", "")
        
        # Split the file name by '_'
        parts = file.split('_')
        
        # Extract the audioname (first three parts)
        audioname = '_'.join(parts[:3])
        
        # Extract the attack_type (remaining parts)
        attack_type = '_'.join(parts[3:])
        
        # Filter only "white" or "cafe" in attack_type
        
        data.append({"audioname":audioname,'adapted': file.split(".")[0], 'attack_type1': attack_type.split("_")[0], 'attack_type2': attack_type.split("_")[1]})

    # Create a DataFrame
    df = pd.DataFrame(data)
    return df

# Example file names
file_names =get_files("/data/ASV19/wav_paired_2")

# Process the file names
result_df = process_file_names(file_names)

# Print the DataFrame
print("Filtered DataFrame:")
print(result_df)


Filtered DataFrame:
          audioname                              adapted attack_type1  \
0      LA_T_1030482  LA_T_1030482_RT09_recompression256k         RT09   
1      LA_T_1021310   LA_T_1021310_RT06_recompression64k         RT06   
2      LA_T_1029007  LA_T_1029007_RT09_recompression320k         RT09   
3      LA_T_1000137   LA_T_1000137_RT06_recompression64k         RT06   
4      LA_T_1004644  LA_T_1004644_RT03_recompression320k         RT03   
...             ...                                  ...          ...   
25374  LA_T_9993186  LA_T_9993186_RT03_recompression128k         RT03   
25375  LA_T_9993979   LA_T_9993979_RT03_recompression16k         RT03   
25376  LA_T_9994240  LA_T_9994240_RT09_recompression196k         RT09   
25377  LA_T_9994911  LA_T_9994911_RT03_recompression128k         RT03   
25378  LA_T_9995068  LA_T_9995068_RT03_recompression320k         RT03   

            attack_type2  
0      recompression256k  
1       recompression64k  
2      recompression32

In [8]:
prot = pd.read_csv("/data/Data/AsvSpoofData_2019/train/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt",sep=" ")
prot.columns = ["tag","audioname","gender","attack","spoof"]

In [9]:
prot_na = prot.merge(result_df,how = "inner",on = "audioname")

In [10]:
prot_na["na"] = "paired"

In [11]:
prot_na[["tag","adapted","gender","attack","spoof","na","attack_type1","attack_type2"]].to_csv(f"/data/ASV19/paired_REV_REC_ASV19.txt",index = False,sep = " ",header= None)

In [25]:
import os
import glob
import pandas as pd

def combine_space_separated_txt_files():
    """
    Prompts the user for an input directory and reads all .txt files (space-separated)
    from that directory, concatenates them into a single DataFrame, and saves
    the combined data to 'combined.txt' in the same directory.
    """
    
    # Prompt user for the input directory
    input_dir = input("Enter the input directory path: ").strip()
    
    # Validate the directory
    if not os.path.isdir(input_dir):
        print(f"ERROR: '{input_dir}' is not a valid directory.")
        return
    
    # Construct the path pattern for all .txt files
    txt_files_pattern = os.path.join(input_dir, "*.txt")
    txt_files = glob.glob(txt_files_pattern)
    
    if not txt_files:
        print(f"No .txt files found in {input_dir}")
        return
    
    df_list = []

    # Read each text file as a space-separated table using pandas
    for file in txt_files:
        df = pd.read_csv(file, sep='\s+', engine='python')
        df_list.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(df_list, ignore_index=True)

    # Define the output file path
    output_file = os.path.join(input_dir, "combined.txt")

    # Save the combined data to a new text file, space-separated
    combined_df.to_csv(output_file, sep=' ', index=False)
    print(f"Combined {len(txt_files)} files into {output_file}.")

if __name__ == "__main__":
    combine_space_separated_txt_files()


<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_3693416/343667983.py:32: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file, sep='\s+', engine='python')


Combined 5 files into /data/ASV19/combined.txt.


In [1]:
import os
from pydub import AudioSegment

def convert_wav_to_flac(folder_path):
    """
    Converts all .wav files in the specified folder to .flac files.
    
    :param folder_path: Path to the folder containing .wav files.
    """
    
    # Ensure folder_path is a valid directory
    if not os.path.isdir(folder_path):
        print(f"The folder path '{folder_path}' is invalid.")
        return
    f_path = "/data/ASV19/flac"
    for filename in os.listdir(folder_path):
        
        # Check if the file has a .wav extension
        if filename.lower().endswith(".wav"):
            wav_path = os.path.join(folder_path, filename)

            # Construct the output .flac file path
            flac_filename = os.path.splitext(filename)[0] + ".flac"
            flac_path = os.path.join(f_path, flac_filename)
            if not os.path.exists(flac_path):
                # Load the .wav file
                audio = AudioSegment.from_wav(wav_path)

                # Export as .flac
                audio.export(flac_path, format="flac")
                print(f"Converted '{filename}' to '{flac_filename}'")

if __name__ == "__main__":
    # Replace with the path to your folder containing the .wav files
    folder_path = "/data/ASV19/wav"
    convert_wav_to_flac(folder_path)
